In [ ]:
%load_ext autoreload
%autoreload 2

import time
import os
import argparse
import cProfile
import pstats
import logging
from datetime import datetime
import matplotlib.pyplot as plt

import numpy as np
import astropy.io.fits as pyfits
import corgihowfsc

from corgihowfsc.utils.howfsc_initialization import get_args, load_files
from corgihowfsc.sensing.DefaultEstimator import DefaultEstimator
from corgihowfsc.sensing.DefaultProbes import DefaultProbes
from corgihowfsc.utils.contrast_nomalization import CorgiNormalization, EETCNormalization
from corgihowfsc.utils.corgisim_gitl_frames import GitlImage


import eetc
from eetc.cgi_eetc import CGIEETC

import howfsc
from howfsc.control.cs import ControlStrategy
from howfsc.control.calcjtwj import JTWJMap

from howfsc.model.mode import CoronagraphMode

from howfsc.util.loadyaml import loadyaml
from howfsc.util.gitl_tools import param_order_to_list

from howfsc.gitl import howfsc_computation
from howfsc.precomp import howfsc_precomputation

from howfsc.scripts.gitlframes import sim_gitlframe

import roman_preflight_proper
### Then, run the following command to copy the default prescription file 
roman_preflight_proper.copy_here()

eetc_path = os.path.dirname(os.path.abspath(eetc.__file__))
howfscpath = os.path.dirname(os.path.abspath(howfsc.__file__))


defjacpath = os.path.join(os.path.dirname(howfscpath), 'jacdata')
defjacpath = r'C:\Users\sredmond\Documents\github_repos\roman-corgi-repos\corgihowfsc\data'
howfscpath = r'C:\Users\sredmond\Documents\github_repos\roman-corgi-repos\corgihowfsc\corgihowfsc' #os.path.dirname(os.path.abspath(corgihowfsc.__file__))


In [ ]:
precomp= 'load_all' if defjacpath is not None else 'precomp_all_once'

current_datetime = datetime.now()
folder_name = 'gitl_simulation_' + current_datetime.strftime("%Y-%m-%d_%H%M%S")
fits_name = 'final_frames.fits'
fileout_path = os.path.join(os.path.dirname(os.path.dirname(corgihowfsc.__file__)), 'data', folder_name, fits_name)

args = get_args(mode='nfov_band1', precomp=precomp, num_process=0, num_threads=1, fileout=fileout_path,jacpath=defjacpath)

# Initialize variables etc

otherlist = []
abs_dm1list = []
abs_dm2list = []
framelistlist = []
scalelistout = []
camlist = []

# User params
niter = args.niter
mode = args.mode
isprof = args.profile
logfile = args.logfile
fracbadpix = args.fracbadpix
nbadpacket = args.nbadpacket
nbadframe = args.nbadframe
fileout = args.fileout
stellar_vmag = args.stellarvmag
stellar_type = args.stellartype
stellar_vmag_target = args.stellarvmagtarget
stellar_type_target = args.stellartypetarget
jacpath = args.jacpath

modelpath, cfgfile, jacfile, cstratfile, probefiles, hconffile, n2clistfiles = load_files(args, howfscpath)


# cfg
cfg = CoronagraphMode(cfgfile)

# Initialize default probes class
probes = DefaultProbes('default')
dm1_list, dm2_list, dmrel_list, dm10, dm20 = probes.get_dm_probes(cfg, probefiles, scalelist=[0.3, 0.3, 0.3, -0.3, -0.3, -0.3])
hconf = loadyaml(hconffile, custom_exception=TypeError)


In [ ]:
probes = DefaultProbes('default')

dm1_list, dm2_list, dmrel_list, dm10, dm20 = probes.get_dm_probes(cfg, probefiles, scalelist=[0.3, 0.3, 0.3, -0.3, -0.3, -0.3])


In [ ]:
cstrat = ControlStrategy(cstratfile)
estimator = DefaultEstimator()

normalization_strategy = CorgiNormalization(cfg, cstrat, hconf, cor=args.mode, corgi_overrides=None, separation_lamD=7)

# imager = corgisimImager()
# imager = compactImager()

imager = GitlImage(
    cfg=cfg,         # Your CoronagraphMode object
    cstrat=cstrat,   # Your ControlStrategy object  
    hconf=hconf,      # Your host config with stellar properties
    backend='corgihowfsc', 
    cor=mode
)

get_cgi_eetc = CGIEETC(mag=hconf['star']['stellar_vmag'],
                   phot='v', # only using V-band magnitudes as a standard
                   spt=hconf['star']['stellar_type'],
                   pointer_path=os.path.join(eetc_path,
                                             hconf['hardware']['pointer']),
)

In [ ]:
_, peakflux = normalization_strategy.calc_flux_rate(get_cgi_eetc, hconf, 1, dm1_list[0], dm2_list[0], 1, gain=1)

im = imager.get_image(dm1_list[0],
                             dm2_list[0],
                             1,
                             gain=1000,
                             crop=False,
                             lind=1,
                             peakflux=peakflux,
                             cleanrow=1024,
                             cleancol=1024,
                             fixedbp=cstrat.fixedbp,
                             wfe=None)

im_norm = normalization_strategy.normalize(im, peakflux, 1)

In [ ]:
# Plot Corgisim image
plt.imshow(np.log10(np.abs(im_norm)))
plt.colorbar()
plt.clim([-10, -4])
plt.title('Corgisim normalized intensity')

In [ ]:
# Get compact image 
nrow = 153
ncol = 153
lrow = 436
lcol = 436
crop = (lrow, lcol, nrow, ncol)

imager_compact = GitlImage(
    cfg=cfg,         # Your CoronagraphMode object
    cstrat=cstrat,   # Your ControlStrategy object  
    hconf=hconf,      # Your host config with stellar properties
    backend='cgi-howfsc', 
    cor=mode
)

normalization_strategy_compact = EETCNormalization()

im_compact = imager_compact.get_image(dm1_list[0],
                             dm2_list[0],
                             1,
                             gain=1000,
                             crop=crop,
                             lind=1,
                             peakflux=peakflux,
                             cleanrow=1024,
                             cleancol=1024,
                             fixedbp=cstrat.fixedbp,
                             wfe=None)

_, peakflux_compact = normalization_strategy_compact.calc_flux_rate(get_cgi_eetc, hconf, 1, dm1_list[0], dm2_list[0], 1, gain=1)

im_compact_norm = normalization_strategy.normalize(im_compact, peakflux_compact, 1)

In [ ]:
# Plot compact image
plt.imshow(np.log10(np.abs(im_compact_norm)))
plt.colorbar()
plt.clim([-10, -4])
plt.title('Compact model normalized intensity')

In [ ]:
# Santity check that DZs are the same size
im_com_norm_cropped = im_compact_norm[1:-1, 1:-1]
im_com_dz = im_com_norm_cropped.copy()
im_com_dz[im_com_norm_cropped==0] = 0
im_com_dz[im_com_norm_cropped<1e-9] = 0
im_com_dz[im_com_norm_cropped>1e-9] = 1

im_dz = im_norm.copy()
im_dz[im_dz<1e-6] = 0
im_dz[im_dz>1e-6] = 1

comb_image = im_com_norm_cropped + im_norm
comb_image[im_com_norm_cropped==0] = 0
comb_image[im_com_norm_cropped<1e-9] = 0

comb_dz = im_com_dz + im_dz
plt.figure(figsize=(8,3))
plt.subplot(121)
plt.imshow(comb_dz)
plt.colorbar()
plt.title('Compact + Corgisim Dark Zone')

plt.subplot(122)
plt.imshow(np.log10(np.abs(comb_image)))
plt.colorbar()
plt.title('Compact + Corgisim Frames')

plt.tight_layout()

In [ ]:
# TODO:
- get normalization thing
- propagate norm additions throught nulling gitl